In [1]:
! pip install ftfy regex tqdm
! pip install git+https://github.com/openai/CLIP.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 2.2 MB/s eta 0:00:00
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-7h79okcx
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-7h79okcx
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=3e20bf7b56c0ac333062904117a7067d77e9f307853c3428e31ab6569f26de33
  Stored in directory: /tmp/pip-ephem-wheel-cache-9qoprhqh/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


In [2]:
import numpy as np
import torch
from pkg_resources import packaging

print("Torch version:", torch.__version__)

Torch version: 2.1.0+cu121


In [3]:
import clip

clip.available_models()

['RN50',
 'RN101',
 'RN50x4',
 'RN50x16',
 'RN50x64',
 'ViT-B/32',
 'ViT-B/16',
 'ViT-L/14',
 'ViT-L/14@336px']

In [4]:
from PIL import Image
import requests

import pandas as pd

from statistics import mean
from sklearn.metrics import confusion_matrix

from transformers import CLIPProcessor, CLIPModel

allergens = {
    0: "gluten",
    1: "eggs",
    2: "milk",
    3: "nuts",
    4: "peanuts",
    5: "soja",
    6: "molluscs",
    7: "fish",
    8: "lupin",
    9: "crustaceans",
    10: "sesame",
    11: "mustard",
    12: "celery",
    13: "sulphites"
}

nb_allergens = len(allergens)

model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")

def predict(image, allergen):
    image = Image.open(image)
    match allergen:
        case 0:
            inputs = processor(text=["ingredient list of a product containing wheat or gluten", "ingredient list of a product without wheat nor gluten"], images=image, return_tensors="pt", padding=True)
        case 1:
            inputs = processor(text=["ingredient list of a product containing eggs", "ingredient list of a product without eggs"], images=image, return_tensors="pt", padding=True)
        case 2:
            inputs = processor(text=["ingredient list of a product containing milk", "ingredient list of a product without milk"], images=image, return_tensors="pt", padding=True)
        case 3:
            inputs = processor(text=["ingredient list of a product containing nuts", "ingredient list of a product without nuts"], images=image, return_tensors="pt", padding=True)
        case 4:
            inputs = processor(text=["ingredient list of a product containing peanuts", "ingredient list of a product without peanuts"], images=image, return_tensors="pt", padding=True)
        case 5:
            inputs = processor(text=["ingredient list of a product containing soya", "ingredient list of a product without soya"], images=image, return_tensors="pt", padding=True)
        case 6:
            inputs = processor(text=["ingredient list of a product containing molluscs", "ingredient list of a product without molluscs"], images=image, return_tensors="pt", padding=True)
        case 7:
            inputs = processor(text=["ingredient list of a product containing fish", "ingredient list of a product without fish"], images=image, return_tensors="pt", padding=True)
        case 8:
            inputs = processor(text=["ingredient list of a product containing lupin", "ingredient list of a product without lupin"], images=image, return_tensors="pt", padding=True)
        case 9:
            inputs = processor(text=["ingredient list of a product containing crustaceans", "ingredient list of a product without crustaceans"], images=image, return_tensors="pt", padding=True)
        case 10:
            inputs = processor(text=["ingredient list of a product containing sesame", "ingredient list of a product without sesame"], images=image, return_tensors="pt", padding=True)
        case 11:
            inputs = processor(text=["ingredient list of a product containing mustard", "ingredient list of a product without mustard"], images=image, return_tensors="pt", padding=True)
        case 12:
            inputs = processor(text=["ingredient list of a product containing celery", "ingredient list of a product without celery"], images=image, return_tensors="pt", padding=True)
        case 13:
            inputs = processor(text=["ingredient list of a product containing sulphites", "ingredient list of a product without sulphites"], images=image, return_tensors="pt", padding=True)

    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image  # this is the image-text similarity score
    probs = logits_per_image.softmax(dim=1)  # we can take the softmax to get the label probabilities

    return probs[0][0].item()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [17]:
from google.colab import files
data_to_load = files.upload()
import io

Saving labellisation(1).xlsx to labellisation(1).xlsx


In [92]:
df = pd.read_excel('/content/labellisation(1).xlsx')

In [93]:
df

,Name,gluten,egg,milk,nuts,peanuts,soya,molluscs,fish,lupin,crustaceans,sesame,mustard,celery,sulphites
0,001.png,1,1,1,1,0,1,0,0,0,0,0,0,0,1
1,002.png,1,1,1,1,0,1,0,0,0,0,0,0,0,0
2,003.png,1,1,1,1,0,1,0,0,0,0,0,0,0,0
3,004.png,1,1,1,1,0,1,0,0,0,0,0,0,0,0
4,005.png,1,1,1,1,0,1,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
214,218.png,0,0,1,0,0,1,0,0,0,0,0,0,0,0
215,219.png,1,0,1,1,0,1,0,0,0,0,0,0,0,0
216,220.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0
217,221.png,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [94]:
df = df.drop('Name',axis='columns')

In [89]:
results = pd.DataFrame()
for image in range(1, 26):
  prediction_array = []
  for allergen in range(0, 14):
            prediction_array.append(predict(f"{image:03d}.jpg", allergen))
  results = results.append(pd.Series(prediction_array, name=f"{image}"))

<ipython-input-89-156f4993bc32>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(pd.Series(prediction_array, name=f"{image}"))
<ipython-input-89-156f4993bc32>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(pd.Series(prediction_array, name=f"{image}"))
<ipython-input-89-156f4993bc32>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(pd.Series(prediction_array, name=f"{image}"))
<ipython-input-89-156f4993bc32>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  results = results.append(pd.Series(prediction_array, name=f"{image}"))
<ipython-input-89-156f4993bc32>:6: F

In [90]:
results

,0,1,2,3,4,5,6,7,8,9,10,11,12,13
1,0.385675,0.169330,0.116219,0.183452,0.226653,0.266286,0.275411,0.274487,0.140998,0.354353,0.185925,0.290245,0.240427,0.108279
2,0.148237,0.059192,0.061426,0.166475,0.146269,0.175701,0.183528,0.148582,0.093173,0.367541,0.204503,0.238817,0.147976,0.078033
3,0.648850,0.418871,0.439551,0.512278,0.496981,0.513282,0.268096,0.435636,0.352049,0.549944,0.429893,0.867408,0.422417,0.499638
4,0.616585,0.462122,0.387110,0.413479,0.454868,0.481388,0.172065,0.350708,0.229586,0.436330,0.323600,0.839013,0.441960,0.359433
5,0.652801,0.566827,0.309575,0.413311,0.495627,0.549897,0.261522,0.454334,0.292942,0.519908,0.371429,0.820597,0.378328,0.501480
6,0.598222,0.310715,0.341645,0.568229,0.542188,0.569838,0.305233,0.438546,0.318808,0.492714,0.524051,0.499279,0.487392,0.302480
7,0.644125,0.408602,0.520943,0.566269,0.523919,0.538450,0.318007,0.377179,0.339792,0.468114,0.584212,0.540777,0.495354,0.267676
8,0.747746,0.412236,0.421118,0.610082,0.682561,0.729461,0.516928,0.503382,0.542832,0.619900,0.719594,0.695006,0.633377,0.410886
9,0.470145,0.186853,0.219079,0.281396,0.172775,0.140549,0.105025,0.267699,0.030324,0.355000,0.227147,0.314646,0.178532,0.036018
10,0.361524,0.227028,0.225409,0.237138,0.269106,0.223375,0.146723,0.189395,0.089700,0.307218,0.185645,0.291566,0.109507,0.121369


In [95]:
df = df.iloc[:25,:]
df

,gluten,egg,milk,nuts,peanuts,soya,molluscs,fish,lupin,crustaceans,sesame,mustard,celery,sulphites
0,1,1,1,1,0,1,0,0,0,0,0,0,0,1
1,1,1,1,1,0,1,0,0,0,0,0,0,0,0
2,1,1,1,1,0,1,0,0,0,0,0,0,0,0
3,1,1,1,1,0,1,0,0,0,0,0,0,0,0
4,1,1,1,1,0,1,0,0,0,0,0,0,0,0
5,0,0,1,1,0,1,0,0,0,0,0,0,0,0
6,0,0,1,1,0,1,0,0,0,0,0,0,0,0
7,0,0,1,1,0,1,0,0,0,0,0,0,0,0
8,1,1,1,1,0,1,0,0,0,0,0,0,0,0
9,1,1,1,1,0,0,0,0,0,0,0,0,0,0


gluten

In [96]:
confusion_matrix(results.iloc[:, 0].round(),df['gluten'],labels=[1,0])

array([[ 7,  4],
       [14,  0]])

egg

In [97]:
confusion_matrix(results.iloc[:, 1].round(),df['egg'],labels=[1,0])

array([[ 1,  0],
       [ 9, 15]])

milk

In [98]:
confusion_matrix(results.iloc[:, 2].round(),df['milk'],labels=[1,0])

array([[ 2,  0],
       [22,  1]])

nuts

In [99]:
confusion_matrix(results.iloc[:, 3].round(),df['nuts'],labels=[1,0])

array([[ 4,  0],
       [10, 11]])

peanuts

In [100]:
confusion_matrix(results.iloc[:, 4].round(),df['peanuts'],labels=[1,0])

array([[ 0,  3],
       [ 2, 20]])

soya

In [101]:
confusion_matrix(results.iloc[:, 5].round(),df['soya'],labels=[1,0])

array([[ 5,  0],
       [14,  6]])

molluscs

In [102]:
confusion_matrix(results.iloc[:, 5].round(),df['molluscs'],labels=[1,0])

array([[ 0,  5],
       [ 0, 20]])

fish

In [103]:
confusion_matrix(results.iloc[:, 6].round(),df['fish'],labels=[1,0])

array([[ 0,  1],
       [ 0, 24]])

lupin

In [104]:
confusion_matrix(results.iloc[:, 7].round(),df['lupin'],labels=[1,0])

array([[ 0,  2],
       [ 0, 23]])

crustaceans

In [105]:
confusion_matrix(results.iloc[:, 8].round(),df['crustaceans'],labels=[1,0])

array([[ 0,  1],
       [ 0, 24]])

sesame

In [106]:
confusion_matrix(results.iloc[:, 9].round(),df['sesame'],labels=[1,0])

array([[ 0,  3],
       [ 2, 20]])

mustard

In [107]:
confusion_matrix(results.iloc[:, 10].round(),df['mustard'],labels=[1,0])

array([[ 0,  3],
       [ 1, 21]])

celery

In [108]:
confusion_matrix(results.iloc[:, 11].round(),df['celery'],labels=[1,0])

array([[ 0,  9],
       [ 0, 16]])

sulphites

In [109]:
confusion_matrix(results.iloc[:, 12].round(),df['sulphites'],labels=[1,0])

array([[ 0,  1],
       [ 3, 21]])